In [ ]:
# 🎯 MODEL REGISTRATION SCRIPT - FIXED & CONFIG DRIVEN

import mlflow
from mlflow.tracking import MlflowClient
import sys
import yaml
import json
import requests
from typing import Dict, Optional, List
from datetime import datetime
from pyspark.sql import SparkSession
import pandas as pd
from delta.tables import DeltaTable

print("=" * 80)
print("🎯 MODEL REGISTRATION SYSTEM - AUTOMATED & DUPLICATE SAFE")
print("=" * 80)

# ---------------------- LOAD PIPELINE CONFIG ----------------------
try:
    with open("pipeline_config.yml", "r") as f:
        pipeline_cfg = yaml.safe_load(f)
    print("✅ pipeline_config.yml loaded")
except Exception as e:
    print(f"❌ Failed to load config: {e}")
    sys.exit(1)


class Config:
    def __init__(self):
        MODEL_TYPE = pipeline_cfg["model"]["type"]
        UC_CATALOG = pipeline_cfg["model"]["catalog"]
        UC_SCHEMA = pipeline_cfg["model"]["schema"]
        BASE_NAME = pipeline_cfg["model"]["base_name"]

        self.MODEL_NAME = f"{UC_CATALOG}.{UC_SCHEMA}.{BASE_NAME}_{MODEL_TYPE}"
        self.EXPERIMENT_NAME = pipeline_cfg["experiment"]["name"]
        self.ARTIFACT_PATH = pipeline_cfg["experiment"]["artifact_path"]
        self.PRIMARY_METRIC = pipeline_cfg["metrics"]["classification"]["primary_metric"]

        self.TOLERANCE = pipeline_cfg["registry"]["duplicate_detection"]["tolerance"]
        self.METRICS_TO_COMPARE = pipeline_cfg["registry"]["duplicate_detection"]["metrics_to_compare"]
        self.DUPLICATE_CHECK_ENABLED = pipeline_cfg["registry"]["duplicate_detection"]["enabled"]

        self.REGISTRATION_LOG_TABLE = pipeline_cfg["tables"]["registration_log"]

        self.SLACK_WEBHOOK = None
        try:
            self.SLACK_WEBHOOK = dbutils.secrets.get("shared-scope", "SLACK_WEBHOOK_URL")
            print("🔐 Slack webhook loaded")
        except:
            print("⚠ Slack webhook NOT configured (Optional)")


config = Config()

print(f"\n📌 Model Registry: {config.MODEL_NAME}")
print(f"📌 Duplicate Logic: {'ENABLED' if config.DUPLICATE_CHECK_ENABLED else 'DISABLED'}")
print(f"📌 Primary Metric: {config.PRIMARY_METRIC}")
print("=" * 80)


# ---------------------- SLACK NOTIFIER ----------------------
class SlackNotifier:
    def __init__(self, webhook_url: Optional[str]):
        self.webhook_url = webhook_url

    def send(self, message: str, level: str = "info"):
        if not self.webhook_url:
            return
        emoji = {"info": "ℹ️", "success": "✅", "warning": "⚠️", "error": "❌"}.get(level, "ℹ️")
        payload = {"text": f"{emoji} {message}"}

        try:
            requests.post(self.webhook_url, json=payload, timeout=5)
        except:
            pass


slack = SlackNotifier(config.SLACK_WEBHOOK)


# ---------------------- INIT SPARK + MLFLOW ----------------------
spark = SparkSession.builder.appName("ModelRegistration").getOrCreate()
mlflow.set_tracking_uri("databricks")
mlflow.set_registry_uri("databricks-uc")
client = MlflowClient()

experiment = mlflow.get_experiment_by_name(config.EXPERIMENT_NAME)
if experiment is None:
    raise Exception(f"Experiment '{config.EXPERIMENT_NAME}' not found!")


# ---------------------- TABLE CREATION ----------------------
def ensure_table_exists(table_name: str, df_schema, spark):
    try:
        spark.sql(f"DESCRIBE TABLE {table_name}")
        print(f"📌 Table exists: {table_name}")
    except:
        print(f"🆕 Creating Delta table: {table_name}")
        empty_df = spark.createDataFrame([], df_schema)
        empty_df.write.format("delta").option("overwriteSchema", "true").saveAsTable(table_name)
        print(f"✅ Table created: {table_name}")


# ---------------------- FETCH RUNS ----------------------
def get_all_runs() -> List[Dict]:
    print("\n📍 Fetching experiment runs...")
    runs = client.search_runs([experiment.experiment_id], order_by=[f"metrics.{config.PRIMARY_METRIC} DESC"], max_results=500)

    return [{
        "run_id": run.info.run_id,
        "run_name": run.info.run_name or "unnamed_run",
        "metrics": {m: run.data.metrics.get(m) for m in config.METRICS_TO_COMPARE if m in run.data.metrics},
        "params": run.data.params,
        "primary_metric": run.data.metrics.get(config.PRIMARY_METRIC),
        "model_uri": f"runs:/{run.info.run_id}/{config.ARTIFACT_PATH}"
    } for run in runs]


# ---------------------- DUPLICATE CHECK ----------------------
def is_duplicate_model(new_model: Dict) -> bool:
    if not config.DUPLICATE_CHECK_ENABLED:
        return False

    versions = client.search_model_versions(f"name='{config.MODEL_NAME}'")

    for version in versions:
        run = client.get_run(version.run_id)

        metrics_match = all(
            abs((run.data.metrics.get(m) or 0) - (new_model["metrics"].get(m) or 0)) <= config.TOLERANCE
            for m in config.METRICS_TO_COMPARE
        )

        if metrics_match and run.data.params == new_model["params"]:
            print(f"⚠ Duplicate model = Matches v{version.version}")
            return True

    return False


# ---------------------- REGISTER MODEL ----------------------
def register_model(model: Dict):
    if is_duplicate_model(model):
        return None

    reg = mlflow.register_model(model["model_uri"], config.MODEL_NAME)
    version = reg.version

    try:
        client.set_model_version_tag(config.MODEL_NAME, version, "run_id", model["run_id"])
        client.set_model_version_tag(config.MODEL_NAME, version, "primary_metric", str(model["primary_metric"]))
    except:
        pass

    print(f"✅ Registered as version: {version}")
    slack.send(f"✅ Registered {config.MODEL_NAME} v{version}", "success")
    return version


# ---------------------- LOGGING DECISION ----------------------
def log_decision(model, registered, version, reason):
    df = pd.DataFrame([{
        "timestamp": datetime.now(),
        "run_id": model["run_id"],
        "run_name": model["run_name"],
        "model_name": config.MODEL_NAME,
        "primary_metric": config.PRIMARY_METRIC,
        "primary_metric_value": model["primary_metric"],
        "metrics_json": json.dumps(model["metrics"]),
        "params_json": json.dumps(model["params"]),
        "registered": registered,
        "registered_version": str(version) if version else None,
        "reason": reason
    }])

    spark_df = spark.createDataFrame(df)

    spark_df = spark_df.withColumn("registered_version", spark_df.registered_version.cast("string"))

    ensure_table_exists(config.REGISTRATION_LOG_TABLE, spark_df.schema, spark)

    delta = DeltaTable.forName(spark, config.REGISTRATION_LOG_TABLE)

    delta.alias("t").merge(
        spark_df.alias("s"),
        "t.run_name = s.run_name AND t.model_name = s.model_name"
    ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

    print("📄 Logged.")


# ---------------------- MAIN ----------------------
def main():
    print("\n🚀 Starting Model Registration...\n")

    runs = get_all_runs()
    if not runs:
        print("❌ No runs found!")
        return

    registered = skipped = 0

    processed_runs = set()  # 🔥 FEEDBACK FIX

    for idx, model in enumerate(runs, start=1):

        # Skip duplicate Slack/log execution for same run_id
        if model['run_id'] in processed_runs:
            continue

        processed_runs.add(model['run_id'])

        print(f"\n{'='*80}\n[{idx}/{len(runs)}] → {model['run_name']} (Metric: {model['primary_metric']})")

        version = register_model(model)

        if version:
            log_decision(model, True, version, "✔ Registered")
            registered += 1
        else:
            slack.send(f"⚠ Duplicate Skipped: {model['run_name']}", "warning")
            log_decision(model, False, None, "⚠ Duplicate Skipped")
            skipped += 1

    print("\n🎉 Completed!\n")
    print(f"✔ Registered: {registered}")
    print(f"⚠ Skipped: {skipped}")


if __name__ == "__main__":
    main()
